In [1]:
import re
import json
import csv
import time

from IPython.display import display
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from requests_html import HTMLSession

from selenium import webdriver
from msedge.selenium_tools import Edge, EdgeOptions

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def scrape(ticker):
  "This opens the edge driver with on a given url"
  #Create the urls
  url_statistics = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
  url_financials = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
  url_historical = f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'


  driver.get(url_financials)
  time.sleep(3)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  pattern = re.compile(r'\s--\sData\s--\s')
  script_data = soup.find('script', text=pattern).contents[0]

  start = script_data.find("context")-2

  #data = [json.loads(line) for line in open('extra.json','r')]
  
  json_data = json.loads(script_data[start:-12])

  json_data['context'].keys()

  annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
  quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

  annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
  quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

  annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
  quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

  qis = pd.json_normalize(quarterly_is)
  qis['ticker'] = ticker
  qis['endDate'] = qis['endDate.fmt']
  qcf = pd.json_normalize(quarterly_cf)
  qcf['ticker'] = ticker
  qbs = pd.json_normalize(quarterly_bs)
  qbs['ticker'] = ticker


  result = pd.merge(qis, qcf, on=["ticker", "endDate.raw"])

  end_result = pd.merge(result, qbs, on=["ticker", "endDate.raw"])

  return end_result

In [4]:
def remove_fluff(df):
    for col_name in df.columns:
        if (col_name[-8:]  == '.longFmt' or col_name[-4:] == '.fmt' or col_name[-2:] == '_x'):
            del df[col_name]
    return df

In [5]:
def scrape_list(list_of_stocks):
    temp = pd.DataFrame()
    for stock in list_of_stocks:
        single_stock_data = scrape(stock)
        temp = pd.concat([temp, single_stock_data])
        temp = remove_fluff(temp)
    df = move_columns(['ticker', 'endDate'], temp)
    return df


In [6]:
def move_columns(list_of_columns, df):
    for column in list_of_columns:
        just_a_column = df[column]
        df = df.drop(columns=[column])
        df.insert(loc=0, column=column, value=just_a_column)
    return df


In [14]:
#Initialize the driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

url = 'https://finance.yahoo.com/'

driver.get(url)


C:\Users\hunte\AppData\Local\Temp\ipykernel_20292\2719000903.py:4: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [ ]:
stocks = ['AAPL', 'AMZN', 'ZM', 'GRBK', 'HP']

data = scrape_list(stocks)

display(data)

In [28]:
new = move_columns(['ticker', 'endDate'], grbk)

display(new)

,endDate.fmt,ticker,maxAge_x,incomeBeforeTax.raw,incomeBeforeTax.fmt,incomeBeforeTax.longFmt,minorityInterest.raw_x,minorityInterest.fmt_x,minorityInterest.longFmt_x,netIncome.raw_x,netIncome.fmt_x,netIncome.longFmt_x,sellingGeneralAdministrative.raw,sellingGeneralAdministrative.fmt,sellingGeneralAdministrative.longFmt,grossProfit.raw,grossProfit.fmt,grossProfit.longFmt,ebit.raw,ebit.fmt,ebit.longFmt,endDate.raw,endDate.fmt_x,operatingIncome.raw,operatingIncome.fmt,operatingIncome.longFmt,incomeTaxExpense.raw,incomeTaxExpense.fmt,incomeTaxExpense.longFmt,totalRevenue.raw,totalRevenue.fmt,totalRevenue.longFmt,totalOperatingExpenses.raw,totalOperatingExpenses.fmt,totalOperatingExpenses.longFmt,costOfRevenue.raw,costOfRevenue.fmt,costOfRevenue.longFmt,totalOtherIncomeExpenseNet.raw,totalOtherIncomeExpenseNet.fmt,totalOtherIncomeExpenseNet.longFmt,netIncomeFromContinuingOps.raw,netIncomeFromContinuingOps.fmt,netIncomeFromContinuingOps.longFmt,netIncomeApplicableToCommonShares.raw,netIncomeApplicableToCommonShares.fmt,netIncomeApplicableToCommonShares.longFmt,maxAge_y,changeToLiabilities.raw,changeToLiabilities.fmt,changeToLiabilities.longFmt,totalCashflowsFromInvestingActivities.raw,totalCashflowsFromInvestingActivities.fmt,totalCashflowsFromInvestingActivities.longFmt,netBorrowings.raw,netBorrowings.fmt,netBorrowings.longFmt,totalCashFromFinancingActivities.raw,totalCashFromFinancingActivities.fmt,totalCashFromFinancingActivities.longFmt,changeToOperatingActivities.raw,changeToOperatingActivities.fmt,changeToOperatingActivities.longFmt,netIncome.raw_y,netIncome.fmt_y,netIncome.longFmt_y,changeInCash.raw,changeInCash.fmt,changeInCash.longFmt,endDate.fmt_y,repurchaseOfStock.raw,repurchaseOfStock.fmt,repurchaseOfStock.longFmt,totalCashFromOperatingActivities.raw,totalCashFromOperatingActivities.fmt,totalCashFromOperatingActivities.longFmt,depreciation.raw,depreciation.fmt,depreciation.longFmt,dividendsPaid.raw,dividendsPaid.fmt,dividendsPaid.longFmt,changeToInventory.raw,changeToInventory.fmt,changeToInventory.longFmt,changeToAccountReceivables.raw,changeToAccountReceivables.fmt,changeToAccountReceivables.longFmt,otherCashflowsFromFinancingActivities.raw,otherCashflowsFromFinancingActivities.fmt,otherCashflowsFromFinancingActivities.longFmt,changeToNetincome.raw,changeToNetincome.fmt,changeToNetincome.longFmt,capitalExpenditures.raw,capitalExpenditures.fmt,capitalExpenditures.longFmt,investments.raw,investments.fmt,investments.longFmt,maxAge,intangibleAssets.raw,intangibleAssets.fmt,intangibleAssets.longFmt,capitalSurplus.raw,capitalSurplus.fmt,capitalSurplus.longFmt,totalLiab.raw,totalLiab.fmt,totalLiab.longFmt,totalStockholderEquity.raw,totalStockholderEquity.fmt,totalStockholderEquity.longFmt,minorityInterest.raw_y,minorityInterest.fmt_y,minorityInterest.longFmt_y,otherCurrentLiab.raw,otherCurrentLiab.fmt,otherCurrentLiab.longFmt,totalAssets.raw,totalAssets.fmt,totalAssets.longFmt,commonStock.raw,commonStock.fmt,commonStock.longFmt,otherCurrentAssets.raw,otherCurrentAssets.fmt,otherCurrentAssets.longFmt,retainedEarnings.raw,retainedEarnings.fmt,retainedEarnings.longFmt,goodWill.raw,goodWill.fmt,goodWill.longFmt,treasuryStock.raw,treasuryStock.fmt,treasuryStock.longFmt,otherAssets.raw,otherAssets.fmt,otherAssets.longFmt,cash.raw,cash.fmt,cash.longFmt,totalCurrentLiabilities.raw,totalCurrentLiabilities.fmt,totalCurrentLiabilities.longFmt,deferredLongTermAssetCharges.raw,deferredLongTermAssetCharges.fmt,deferredLongTermAssetCharges.longFmt,propertyPlantEquipment.raw,propertyPlantEquipment.fmt,propertyPlantEquipment.longFmt,totalCurrentAssets.raw,totalCurrentAssets.fmt,totalCurrentAssets.longFmt,longTermInvestments.raw,longTermInvestments.fmt,longTermInvestments.longFmt,netTangibleAssets.raw,netTangibleAssets.fmt,netTangibleAssets.longFmt,netReceivables.raw,netReceivables.fmt,netReceivables.longFmt,longTermDebt.raw,longTermDebt.fmt,longTermDebt.longFmt,inventory.raw,inventory.fmt,inventory.longFmt,accountsPayable.raw,accountsPayable.